In [1]:
# Turn off warnings and info messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

2024-12-04 15:10:59.305320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733343059.328043    5175 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733343059.334242    5175 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# incrementally use GPU memory as needed instead of allocating all at once
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
# Set seed
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Get Dataset


In [4]:
import urllib.request
import zipfile

if not os.path.exists('data'):
    url = 'https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip'
    zip_file_name = 'data/data.zip'
    os.makedirs(os.path.dirname(zip_file_name), exist_ok=True)
    urllib.request.urlretrieve(url, zip_file_name)
    
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(".")
    os.remove(zip_file_name)
    print("Downloaded data")
else:
    print("Data folder 'data' already exists. Skipping download")

Downloaded data


### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(200, 200, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

In [5]:
inputs = keras.Input(shape=(200, 200, 3))

x = keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = keras.layers.MaxPooling2D((2, 2))(x)

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(64, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.models.Model(inputs=inputs, outputs=outputs)
sgd_opt = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
loss = keras.losses.BinaryCrossentropy(from_logits=False)
model.compile(optimizer=sgd_opt, loss=loss, metrics=['accuracy'])

I0000 00:00:1733343079.345134    5175 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9702 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that.

* 896
* 11214912
* 15896912
* 20072512

In [6]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

### Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_gen = ImageDataGenerator(rescale=1./255)

In [9]:
train_ds = train_gen.flow_from_directory('data/train',target_size=(200, 200), batch_size=20, shuffle=True, class_mode = "binary")

Found 800 images belonging to 2 classes.


In [10]:
test_gen = ImageDataGenerator(rescale=1./255)

In [11]:
test_ds = test_gen.flow_from_directory('data/test',target_size=(200, 200), batch_size=20, shuffle=True, class_mode = "binary")

Found 201 images belonging to 2 classes.


In [12]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

/home/bullsvsbears/pycharmenvs/tf-gpu/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1733343082.501390    5227 service.cc:148] XLA service 0x7f387c007ff0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733343082.501453    5227 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1080 Ti, Compute Capability 6.1
I0000 00:00:1733343082.663110    5227 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/40 ━━━━━━━━━━━━━━━━━━━━ 2:06 3s/step - accuracy: 0.4500 - loss: 0.6905

I0000 00:00:1733343084.427947    5227 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 367ms/step - accuracy: 0.5825 - loss: 0.6985 - val_accuracy: 0.5622 - val_loss: 0.6885
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 304ms/step - accuracy: 0.7019 - loss: 0.5631 - val_accuracy: 0.6418 - val_loss: 0.6178
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 297ms/step - accuracy: 0.7207 - loss: 0.5375 - val_accuracy: 0.6517 - val_loss: 0.6108
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 294ms/step - accuracy: 0.7672 - loss: 0.5124 - val_accuracy: 0.5920 - val_loss: 0.7512
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 306ms/step - accuracy: 0.7404 - loss: 0.5030 - val_accuracy: 0.6766 - val_loss: 0.5918
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 307ms/step - accuracy: 0.7789 - loss: 0.4817 - val_accuracy: 0.6468 - val_loss: 0.6320
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 310ms/step - accuracy: 0.7668 - loss: 0.4634 - val_accuracy: 0.6716 - val_loss: 0.5870
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 302ms/step - accuracy: 0.8258 - loss: 0.4021 - val_accuracy: 0.671

### Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.10
* 0.32
* 0.50
* 0.72

In [13]:
np.median(history.history['accuracy']).round(2)

np.float64(0.76)

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.028
* 0.068
* 0.128
* 0.168

In [14]:
np.std(history.history['loss']).round(3)

np.float64(0.088)

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=50,`
* `width_shift_range=0.1,`
* `height_shift_range=0.1,`
* `zoom_range=0.1,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [15]:
train_gen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_gen.flow_from_directory(
    'data/train',target_size=(200, 200), 
    batch_size=20, 
    shuffle=True, 
    class_mode = "binary"
)

Found 800 images belonging to 2 classes.


### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
> **Note:** make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.26
* 0.56
* 0.86
* 1.16

In [16]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 20s 485ms/step - accuracy: 0.6704 - loss: 0.6041 - val_accuracy: 0.7114 - val_loss: 0.5697
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 478ms/step - accuracy: 0.6667 - loss: 0.5827 - val_accuracy: 0.7264 - val_loss: 0.5638
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 473ms/step - accuracy: 0.7012 - loss: 0.5880 - val_accuracy: 0.5771 - val_loss: 0.6674
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 460ms/step - accuracy: 0.6781 - loss: 0.5869 - val_accuracy: 0.6617 - val_loss: 0.6318
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 22s 491ms/step - accuracy: 0.6763 - loss: 0.5715 - val_accuracy: 0.6716 - val_loss: 0.6034
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 474ms/step - accuracy: 0.7087 - loss: 0.5581 - val_accuracy: 0.7612 - val_loss: 0.5328
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 473ms/step - accuracy: 0.7228 - loss: 0.5503 - val_accuracy: 0.7164 - val_loss: 0.5606
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 19s 484ms/step - accuracy: 0.7009 - loss: 0.5682 - val_accu

In [17]:
np.mean(history.history['val_loss']).round(2)

np.float64(0.58)

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

* 0.31
* 0.51
* 0.71
* 0.91

In [18]:
np.mean(history.history['val_accuracy'][5:]).round(2)

np.float64(0.73)